In [2]:
import mysql.connector
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [4]:
db_user = os.environ.get('my_username')
db_password = os.environ.get('my_password')
db = mysql.connector.connect(host= "localhost", user = db_user, passwd = db_password, database = "data_bank")
db

# A. Customer Nodes Exploration

In [6]:
# How many unique nodes are there on the Data Bank system?

a1 = pd.read_sql_query("select count(distinct node_id) from customer_nodes",db)
a1

,count(distinct node_id)
0,5


In [7]:
# What is the number of nodes per region?
a2 =pd.read_sql_query("""select r.region_name, r.region_id, count( cn.node_id) 
from customer_nodes cn
inner join regions r on cn.region_id = r.region_id
group by cn.region_id 
order by cn.region_id""",db)
a2

,region_name,region_id,count( cn.node_id)
0,Australia,1,770
1,America,2,735
2,Africa,3,714
3,Asia,4,665
4,Europe,5,616


In [8]:
# How many customers are allocated to each region?

a3 = pd.read_sql_query("""select r.region_name,r.region_id, count(cn.customer_id)
from customer_nodes cn
inner join regions r on cn.region_id = r.region_id
group by cn.region_id 
order by cn.region_id""",db)
a3

,region_name,region_id,count(cn.customer_id)
0,Australia,1,770
1,America,2,735
2,Africa,3,714
3,Asia,4,665
4,Europe,5,616


In [9]:
# How many days on average are customers reallocated to a different node?

a4 =pd.read_sql_query("""with table1 as (SELECT *,end_date-start_date as diff 
FROM data_bank.customer_nodes 
where end_date != '9999-12-31' 
GROUP BY customer_id, node_id, start_date, end_date
order by customer_id , node_id),
table2 as (SELECT customer_id, node_id, SUM(table1.diff) AS sum_diff
FROM table1
GROUP BY customer_id, node_id)
select round(avg(table2.sum_diff),2) as avg_cust_reallocated_newNode from table2""",db)
a4

,avg_cust_reallocated_newNode
0,76.56


In [ ]:
# What is the median, 80th and 95th percentile for this same reallocation days metric for each region? 

# UPDATING

# B. Customer Transactions

In [10]:
# What is the unique count and total amount for each transaction type?

b1 = pd.read_sql_query("select txn_type, count(txn_type) from customer_transactions group by txn_type",db)
b1

,txn_type,count(txn_type)
0,deposit,2671
1,withdrawal,1580
2,purchase,1617


In [ ]:
# What is the average total historical deposit counts and amounts for all customers?

# UPDATING

In [ ]:
# For each month - how many Data Bank customers make more than 1 deposit and either 1 purchase or 1 withdrawal in a single month?

b4 = pd.read_sql_query("""SELECT 
    customer_id,
    DATE_FORMAT(txn_date, '%M') as month_name,
    SUM(CASE WHEN txn_type = 'deposit' THEN 0 ELSE 1 END) AS deposit_count,
    SUM(CASE WHEN txn_type = 'purchase' THEN 0 ELSE 1 END) AS purchase_count,
    SUM(CASE WHEN txn_type = 'withdrawal' THEN 1 ELSE 0 END) AS withdrawal_count
  FROM customer_transactions
  GROUP BY customer_id, month_name""",db)
b4


# UPDATING